### Machine Learning Regresion Lineal
Voy a usar un modelo simple y explicable para ver cuánto influyen edad, BMI, hijos y hábitos de vida en el coste médico

Setup librerias

In [78]:
import numpy as np
import pandas as pd
import json
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
from sklearn.exceptions import ConvergenceWarning

sns.set(style="whitegrid")
pd.set_option("display.max_columns", None)

Seleccion del mejor Dataset

In [79]:
X_train_CON_outliers      = pd.read_csv("../data/processed/X_train_CON_outliers.xlsx")
X_train_CON_outliers_norm = pd.read_csv("../data/processed/X_train_CON_outliers_norm.xlsx")
X_train_CON_outliers_scal = pd.read_csv("../data/processed/X_train_CON_outliers_scal.xlsx")
X_train_SIN_outliers      = pd.read_csv("../data/processed/X_train_SIN_outliers.xlsx")
X_train_SIN_outliers_norm = pd.read_csv("../data/processed/X_train_SIN_outliers_norm.xlsx")
X_train_SIN_outliers_scal = pd.read_csv("../data/processed/X_train_SIN_outliers_scal.xlsx")

y_train = pd.read_csv("../data/processed/y_train.xlsx")


#Utilizaremos el modelo de regresion logistica
datasets = [X_train_CON_outliers,
    X_train_CON_outliers_norm,
    X_train_CON_outliers_scal,
    X_train_SIN_outliers,
    X_train_SIN_outliers_norm,
    X_train_SIN_outliers_scal
    ]

models = []
metrics = []

for dataset in datasets:

    model = LinearRegression()
    model.fit(dataset, y_train)
    y_pred = model.predict(dataset)
    metric = model.score(dataset, y_train) # Calculado por coeficiente de determinacion
    metrics.append(metric)
    models.append(model)


best_metric = max(metrics)
best_index = metrics.index(best_metric)
print(f"El mejor dataset es: \n{datasets[best_index]}")
print(f"Error cuadrático medio: {mean_squared_error(y_train, y_pred)}")
print(f"Coeficiente de determinación: {metric}")

El mejor dataset es: 
      age  sex_n     bmi  children  smoker_n  region_n
0      31      1  25.900         3         0         0
1      44      0  32.340         1         1         1
2      18      1  34.100         0         1         1
3      28      1  37.100         1         1         0
4      42      0  25.300         1         1         0
...   ...    ...     ...       ...       ...       ...
1064   24      0  29.925         0         1         2
1065   55      1  21.500         1         1         0
1066   51      0  25.800         1         1         0
1067   18      0  30.305         0         1         3
1068   38      0  19.950         2         1         3

[1069 rows x 6 columns]
Error cuadrático medio: 36736260.33152311
Coeficiente de determinación: 0.7540090223124195


Tras entrenar un modelo de regresion lineal sobre los distintos conjuntos de datos generados, se observa que el DF compuesto por ge, sex_n, bmi, children, smoker_n y region_n, presenta el mejor rendimiento. Tiene un accuracy del 75%.

Regularizacion

In [80]:
diccionario_modelos = {
    "lasso_model": {
        "model": Lasso(random_state=10),
        "params": {
                       "fit_intercept": [True, False],
                       "alpha": [0.001, 0.01, 0.1, 1, 10, 100],
                       "max_iter":[100, 300, 400, 500],
                       "selection": ["cyclic", "random"],
                       "tol":[00.1, 0.0001, 0.000001]
                  }
    },

    "ridge_model": {
        "model": Ridge(random_state=10),
        "params": {
                       "fit_intercept": [True, False],
                       "alpha": [0.001, 0.01, 0.1, 1, 10, 100],
                       "max_iter":[100, 300, 400, 500],
                       "tol":[00.1, 0.0001, 0.000001],
                       "solver": ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga", "lbfgs"]
                  }
    },

    "elasticNet_model": {
        "model": ElasticNet(random_state=10),
        "params": {
                       "fit_intercept": [True, False],
                       "alpha": [0.001, 0.01, 0.1, 1, 10, 100],
                       "max_iter":[100, 300, 400, 500],
                       "l1_ratio": [0, 0.5, 1],
                       "fit_intercept": [True, False],
                       "selection": ["cyclic", "random"],
                       "tol":[00.1, 0.0001, 0.000001]
                  }
    }                        
}

Se emplearon modelos de regresion regularizaa junto con busquedad exhaustiva de hiperparametros para seleccionar la configuracion que mejor generaliza, reduciendo el riesgo de sobreajuste

Grid search

In [81]:
from sklearn.model_selection import GridSearchCV
import numpy as np

best_scored_model = -np.inf
best_model_gr = None
name_model = ""
best_params = None

X_train_best = datasets[best_index]
y_train_vec = np.ravel(y_train)

for name, info in diccionario_modelos.items():
    grid = GridSearchCV(
        estimator=info["model"],
        param_grid=info["params"],
        scoring="r2",
        cv=5,
        n_jobs=-1
    )
    grid.fit(X_train_best, y_train_vec)

    if grid.best_score_ > best_scored_model:
        best_scored_model = grid.best_score_
        best_model_gr = grid.best_estimator_
        name_model = name
        best_params = grid.best_params_



/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:716: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.378e+09, tolerance: 2.834e+07
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:716: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.184e+10, tolerance: 2.605e+07
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:716: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1

In [82]:
print("El mejor modelo es:", name_model)
print("Mejores hiperparámetros:", best_params)
print("Mejor R² (CV):", best_scored_model)

El mejor modelo es: lasso_model
Mejores hiperparámetros: {'alpha': 100, 'fit_intercept': True, 'max_iter': 100, 'selection': 'cyclic', 'tol': 0.1}
Mejor R² (CV): 0.7451917128336627


El modelo Lasso con regularización fuerte mostró el mejor equilibrio entre rendimiento y simplicidad, proporcionando un modelo más estable sin pérdida significativa de capacidad predictiva.

Random search


In [83]:
from sklearn.model_selection import RandomizedSearchCV

best_scored_model = 0
best_model_gr = ""
name_model = ""

for name, info in diccionario_modelos.items():
    grid = RandomizedSearchCV(info["model"], info["params"], scoring="r2", n_iter = 30, n_jobs = -1)
    grid.fit(datasets[best_index], y_train)

    if best_scored_model < grid.best_score_:
        best_scored_model = grid.best_score_
        best_model_rnd = grid.best_estimator_
        name_model = name

/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/sklearn/model_selection/_valida

In [84]:
print("El mejor modelo es: ", name_model)
print("El mejor estimador es: ", best_model_rnd)
print("La mejor puntuación es: ", best_scored_model)

El mejor modelo es:  lasso_model
El mejor estimador es:  Lasso(alpha=100, max_iter=100, random_state=10, tol=0.1)
La mejor puntuación es:  0.7451917128336627


Métricas finales

In [ ]:
#Volvemos a entrenar el modelo con el mejor dataset de x_train y x_test


metric_train = mse_train
metric_test = mse_test
metric_R2_train = r2_train
metric_R2_test = r2_test


# --- TRAIN ---
X_train_CON_outliers      = pd.read_csv("../data/processed/X_train_CON_outliers.xlsx")
X_train_CON_outliers_norm = pd.read_csv("../data/processed/X_train_CON_outliers_norm.xlsx")
X_train_CON_outliers_scal = pd.read_csv("../data/processed/X_train_CON_outliers_scal.xlsx")
X_train_SIN_outliers      = pd.read_csv("../data/processed/X_train_SIN_outliers.xlsx")
X_train_SIN_outliers_norm = pd.read_csv("../data/processed/X_train_SIN_outliers_norm.xlsx")
X_train_SIN_outliers_scal = pd.read_csv("../data/processed/X_train_SIN_outliers_scal.xlsx")

y_train = pd.read_csv("../data/processed/y_train.xlsx").squeeze()

datasets_train = [
    X_train_CON_outliers,
    X_train_CON_outliers_norm,
    X_train_CON_outliers_scal,
    X_train_SIN_outliers,
    X_train_SIN_outliers_norm,
    X_train_SIN_outliers_scal
]

# --- TEST ---
X_test_CON_outliers      = pd.read_csv("../data/processed/X_test_CON_outliers.xlsx")
X_test_CON_outliers_norm = pd.read_csv("../data/processed/X_test_CON_outliers_norm.xlsx")
X_test_CON_outliers_scal = pd.read_csv("../data/processed/X_test_CON_outliers_scal.xlsx")
X_test_SIN_outliers      = pd.read_csv("../data/processed/X_test_SIN_outliers.xlsx")
X_test_SIN_outliers_norm = pd.read_csv("../data/processed/X_test_SIN_outliers_norm.xlsx")
X_test_SIN_outliers_scal = pd.read_csv("../data/processed/X_test_SIN_outliers_scal.xlsx")

y_test = pd.read_csv("../data/processed/y_test.xlsx").squeeze()

datasets_test = [
    X_test_CON_outliers,
    X_test_CON_outliers_norm,
    X_test_CON_outliers_scal,
    X_test_SIN_outliers,
    X_test_SIN_outliers_norm,
    X_test_SIN_outliers_scal
]

# --- Entrenamiento final con el mejor dataset ---
best_index = best_index  

model_f = Lasso(alpha=100, max_iter=100, random_state=10, tol=0.1)
model_f.fit(datasets_train[best_index], y_train)

# Train
pred_train = model_f.predict(datasets_train[best_index])
mse_train = mean_squared_error(y_train, pred_train)
r2_train  = r2_score(y_train, pred_train)

# Test
pred_test = model_f.predict(datasets_test[best_index])
mse_test = mean_squared_error(y_test, pred_test)
r2_test  = r2_score(y_test, pred_test)

print(" TRAIN -> R²:", r2_train, "| MSE:", mse_train)
print(" TEST  -> R²:", r2_test,  "| MSE:", mse_test)

📌 TRAIN -> R²: 0.7533688031026231 | MSE: 36831870.584311314
📌 TEST  -> R²: 0.73677736020511 | MSE: 35630475.491553836


In [86]:
print(f"La mejor métrica de nuestros datasets x_train son: {metric_train} y los x_test son: {metric_test}")
print(f"El mejor coeficiente de determinacion de nuestros datasets x_train son: {metric_R2_train} y los x_test son: {metric_R2_test}")

La mejor métrica de nuestros datasets x_train son: 36831870.584311314 y los x_test son: 35630475.491553836
El mejor coeficiente de determinacion de nuestros datasets x_train son: 0.7533688031026231 y los x_test son: 0.73677736020511


El modelo Lasso optimizado presenta un rendimiento consistente entre entrenamiento y test, lo que indica una buena capacidad de generalización. La diferencia moderada entre las métricas de ambos conjuntos sugiere que la regularización aplicada ha permitido controlar el sobreajuste sin comprometer significativamente el poder predictivo del modelo.